## Подбор еды с помощью LangGraph Agentic Workflow

Рассмотрим сценарий, когда нам нужно подобрать меню для пользователя в некотором ресторане, таким образом, чтобы вино подходило к основному блюду. Для этой задачи можно использовать типовой процесс из реальной жизни, в котором участвуют:
* Хостесс для выяснения первоначальных предпочтений посетителя
* Официант, знакомый с меню ресторана, и умеющий отвечать на вопросы по блюдам
* Сомелье, умеющий подбирать подходящее вино

Для каждой из этих задач можем использовать отдельного агента со своими навыками (инструментами) и/или RAG-базой знаний.

Для создания агентов используем код из [рассмотренного ранее примера](advanced-assistant.ipynb). Для начала - несколько полезных функций.

In [1]:
import os
from IPython.display import Markdown, display

if os.name == 'nt':
    os.environ["GRPC_POLL_STRATEGY"] = "poll"
    import asyncio
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

def fread(fn):
    with open(fn, encoding='utf-8') as f:
        return f.read()
    
def printx(x):
    display(Markdown(x))

## Авторизация

Для работы с языковыми моделями нам понадобится авторизоваться в Yandex Cloud. Это можно сделать несколькими способами:

* через iam-токен. Для получения iam-токена необходимо создать авторизованный ключ доступа к сервисному аккаунту, и знать `service_accound_id`, `key_id` и `private_key`. Скачайте файл с ключами доступа `authorized_key.json`, и добавьте к нему поле `folder_id`
* **[рекомендованный способ]** через ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere или в переменных окружения. Если вы используете набор переменных окружения в файле `.env` - запустите следующую ячейку для их загрузки.

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [2]:
import json
import os
from util.iam_auth import get_iam

if os.path.exists('authorized_key.json'):
    with open('authorized_key.json') as f:
        auth_key = json.load(f)

    api_key = get_iam(auth_key['service_account_id'],auth_key['id'],auth_key['private_key'])
    folder_id = auth_key['folder_id']
    print(f"Using IAM Token Auth with folder_id={folder_id}")
else:
    folder_id = None

if folder_id is None:
    folder_id = os.environ["folder_id"]
    api_key = os.environ["api_key"]

Using IAM Token Auth with folder_id=b1gst3c7cskk2big5fqn


Создадим какую-нибудь модель из Foundation Models и убедимся, что она кое-что знает про вина. 

> ВНИМАНИЕ: Для правильной работы необходимо передать `folder_id` в параметр `project` при создании объекта OpenAI SDK.

In [3]:
import os
from openai import OpenAI,AsyncOpenAI

model = f"gpt://{folder_id}/yandexgpt/rc"
model = f"gpt://{folder_id}/gemma-3-27b-it/latest"
model = f"gpt://{folder_id}/gpt-oss-120b/latest"
model = f"gpt://{folder_id}/qwen3-235b-a22b-fp8/latest"

client = OpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

aclient = AsyncOpenAI(
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    api_key=api_key,
    project=folder_id
)

Теперь создадим основной класс `Agent`, который позволит нам создавать агента, передавая ему:
* Системный промпт `instruction`
* Набор внешних инструментов `tools`
* Набор документов для RAG `search_content`

Агент при создании автоматически создаст поисковый индекс из документов, а при вызове отработает Function Calling и вернет ответ.

In [47]:
import io

class Agent():

    def __init__(self,
            name,
            instruction, 
            tools = [], search_content = [], 
            model = model,
            response_format = None
            ):
        self.user_sessions = {}
        self.name = name
        self.instruction = instruction
        self.model = model
        self.tool_map = { x.__name__ : x for x in tools if issubclass(x, BaseModel) }
        self.tools = [
            self._create_tool_annot(x) for x in tools
        ]
        self.response_format = response_format
        self.vector_store = None
        if search_content:
            i=0
            self.vector_store = client.vector_stores.create(name=f'rag_store_{self.name}')
            for c in search_content:
                f = client.files.create(
                        purpose="assistants",
                        file = (f'rag_{self.name}_{i}.txt',io.BytesIO(c.encode("utf-8")),'text/markdown'))
                client.vector_stores.files.create(file_id=f.id, vector_store_id=self.vector_store.id)
                print(f" + Uploading rag_{self.name}_{i}.txt as id={f.id} to store={self.vector_store.id}")
                i+=1
            self.tools.append({
                "type" : "file_search",
                "vector_store_ids" : [self.vector_store.id],
                "max_num_results" : 5,
            })
            
    def _create_tool_annot(self, x):
        if issubclass(x, BaseModel):
            return {
                "type": "function",
                "name": x.__name__,
                "description": x.__doc__,
                "parameters": x.model_json_schema(),
            }
        else:
            return x

    def __call__(self, message, session_id='default',return_raw=False):
        s = self.user_sessions.get(session_id,{ 'previous_response_id' : None, 'history' : [] })
        s['history'].append({ 'role': 'user', 'content': message })
        txt = None
        if self.response_format:
            txt = {
                "format" : {
                    "type" : "json_schema",
                    "name" : "struct_out",
                    "schema" : self.response_format.model_json_schema()
                }
            }
        res = client.responses.create(
            model = self.model,
            store = True,
            tools = self.tools,
            instructions = self.instruction,
            previous_response_id = s['previous_response_id'],
            input = message,
            text = txt
        )
        # Обрабатываем вызов локальных инструментов
        tool_calls = [item for item in res.output if item.type == "function_call"]
        if tool_calls:
            s['history'].append({ 'role' : 'func_call', 'content' : res.output_text })
            out = []
            for call in tool_calls:
                print(f" + Обрабатываем: {call.name} ({call.arguments})")
                try:
                    fn = self.tool_map[call.name]
                    obj = fn.model_validate(json.loads(call.arguments))
                    result = obj.process(session_id)
                except Exception as e:
                    result = f"Ошибка: {e}"
                #print(f" + Результат: {result}")
                out.append({
                    "type": "function_call_output",
                    "call_id": call.call_id,
                    "output": result
                })
                res = client.responses.create(
                    model=self.model,
                    input=out,
                    tools=self.tools,
                    previous_response_id=res.id,
                    store=True
                )
        # MCP Approval Requests
        mcp_approve = [ item for item in res.output if item.type == "mcp_approval_request"]
        if mcp_approve:
            res = client.responses.create(
                model=self.model,
                previous_response_id=res.id,
                tools = self.tools,
                input=[{
                    "type": "mcp_approval_response",
                    "approve": True,
                    "approval_request_id": m.id
                }
                for m in mcp_approve
                ])
        s['previous_response_id'] = res.id
        s['history'].append({ 'role' : 'assistant', 'content' : res.output_text })
        self.user_sessions[session_id] = s
        if return_raw:
            return res
        if self.response_format:
            return self.response_format.model_validate_json(res.output_text)
        else:
            return res.output_text

    def history(self, session_id='default'):
        return self.user_sessions[session_id]['history']

Для решения нашей задачи будем моделировать поход в ресторан в реальной жизни. У нас будут следующие роли:

* Агент-официант владеет информацией о меню ресторана, и может рекомендовать конкретное блюдо
* Агент-сомелье умеет подбирать сочетания блюд и вина
* Агент-хостесс принимает первоначальные предпочтения посетителя, и далее маршрутизирует диалог между другими агентами для решения задачи.

Создадим этих агентов:

## Агент-официант

Для начала, создадим агента, который владеет меню нашего ресторана. Поскольку меню обычно невелико по объему, то закинем его в конекст агента:

In [29]:
prompt = f"""
Ты - официант в ресторане, который должен советовать посетителям блюда и отвечать 
на вопросы по меню. Меню приведено ниже в тройных обратных кавычках. Если в меню нет указанного блюда
 - напиши, что блюдо отсутствует в меню. Не придумывай ничего!
Меню:
```
{fread('data/menu/food.md')}
{fread('data/menu/drinks.md')}
``` 
"""

waiter = Agent(
    'waiter',
    instruction=prompt)  

In [31]:
printx(waiter("Сколько стоит ливерная колбаса?"))

Ливерная колбаса отсутствует в меню.

In [32]:
printx(waiter("Какое самое дорогое вино?"))

Самое дорогое вино — **Пино Нуар** от **Domaine de la Romanée-Conti, Франция**, 2017 года, цена за бокал — **5200 рублей**.

In [33]:
printx(waiter("Какой самый дорогой стейк в меню?"))

Самый дорогой стейк в меню — **"Бык на взводе"**, цена — **2500 рублей**.

## Агент-сомелье

Теперь создадим агента, который может дать рекомендации по сочетанию вин и еды. Будем использовать таблицу соответствий еды и вина такого вида:

Блюдо, к которому надо подобрать вино | Вино, которое подходит к этому блюду
--------|--------
Баклажаны, запеченые с сыром | Красное вино: «среднетелые»* сухие — Гренаш (Гарнача), Санджовезе (Кьянти), Карменер, Менсия, молодые Темпранильо, легкотелое Мерло.
Баранина деликатесная (филе или каре ягненка) | Красное вино: сухие выдержанные вина из винограда Пино Нуар, Менсия, Неббиоло (в том числе элегантные выдержанные Бароло и Барбареско), Гамэ (элегантные бургундские Божоле Виляж).
Баранина пикантная: жареная, гриль, тушеная — со специями | Красные вина: сухие вина из винограда Каберне Совиньон, «ронские»** ассамбляжи Гренаш+Сира+Мурведр, французский Мальбек, немного «скругленная» Барбера, Сира (Шираз). Выдержанные вина из Санджовезе (Кьянти Классико, вина Монтальчино), Альянико, «супертосканские»*** вина, добротные Crianza Риохи. Примитиво и Зинфандель. Саперави из России.

Для этого также используем RAG, как в предыдущем примере, только вручную нарежем табличку на небольшие фрагменты, обязательно содержащие в себе заголовок:

In [ ]:
with open("data/food_wine_table.md", encoding="utf-8") as f:
    food_wine = f.readlines()
header = food_wine[:2]
chunk_size = 1000 * 3  # approx 1000 tokens * 3 char/token
docs = []
s = header.copy()
for x in food_wine[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        docs.append("".join(s))
        s = header.copy()

In [35]:
prompt = """
Ты - опытный сомелье, который должен рекомендовать клиенту правильные сочетания блюд и вина. Отвечай на просьба подобрать вино к еде или еду к вину, используя имеющуюся у тебя информацию. Также если
пользователь просит посоветовать ему какое-то интересное сочетание - сделай это на основе имеющихся данных. Не придумывай ничего!
"""

sommelier = Agent(
    'sommelier',
    search_content=docs,
    instruction=prompt)  

 + Uploading rag_sommelier_0.txt as id=fvtufb91gkfr47ffpcsh to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_1.txt as id=fvtidf78bglfnq7fgfsu to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_2.txt as id=fvthg8104nsgrq8uedjq to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_3.txt as id=fvtt7i3fa26msaqnh3d7 to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_4.txt as id=fvt0ao41eanhnrivppsl to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_5.txt as id=fvt2bv8mb6ci0a44ugl1 to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_6.txt as id=fvt87g7k2qbv3f4kns1r to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_7.txt as id=fvtuh90h9uulufq3d4lk to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_8.txt as id=fvtpl3j79bfh2vas933i to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_9.txt as id=fvtji6t7sfifvqm5mtnk to store=fvtcmqftpc3i1r0608jj
 + Uploading rag_sommelier_10.txt as id=fvtqc000aiuu7l40a7g5 to store=fvtcmqftpc3i1r0608jj
 + Upload

In [36]:
printx(sommelier("Какое вино подойдёт к стейку?"))

Для стейка отлично подойдёт **Каберне Совиньон** от **Château de Parenchère, Франция**, 2017 года. Это вино обладает насыщенным танинным каркасом, яркой кислотностью и богатым букетом, в котором переплетаются оттенки спелой вишни, табака, дуба и чёрного перца. Такое сочетание идеально дополняет вкус стейка, подчёркивая его сочность и глубину.

In [37]:
printx(sommelier("Какое вино подойдёт к стейку филе миньон?"))

К стейку **филе миньон** идеально подойдёт **Пино Нуар** от **Domaine de la Romanée-Conti, Франция**, 2017 года. Это вино обладает утончённой структурой, нежной танинностью и сложным ароматическим букетом, включающим ноты вишни, красной смородины, фиалки и лёгкие древесные оттенки. Такое сочетание подчеркнёт нежность и изысканность филе миньон, не перебивая его тонкий вкус.

## Агент-хостесс

На входе в ресторан посетителя приветствует хостесс, которая интересуется, что именно хочет есть человек. Для этого заведём специального агента, который будет вычленять из запроса пользователя его предпочтения по вину и по еде. Для этого используем структурный вывод модели.

In [38]:
from pydantic import BaseModel

class PersonPref(BaseModel):
    food_pref : str = ""
    wine_pref : str = ""
    
prompt = """
Ты - официант в ресторане, задача которого принять заказ у посетителя.
Тебе необходимо выслушать его и понять, есть ли у него какие-то предпочтения
по еде или по вину. Верни ответ в формате json, с полями 
`food_pref` и `wine_pref`, в которых будут указаны предпочтения. 
Если нет предпочтений, то верни пустые строки.
"""

hostess = Agent(
    'hostess',
    instruction=prompt,
    response_format=PersonPref)

In [39]:
res = hostess("Привет, я хочу поесть что-то из мяса")
res

PersonPref(food_pref='мясо', wine_pref='')

## Собираем агентов вместе: LangGraph

Для объединения агентов вместе будем использовать фреймворк LangGraph. В основе этого фреймворка - идея графа состояний (state machine) и **состояния**, в зависимости от которого определяется дальнейшее поведение системы.

Узлами графа являются функции, которые возвращают **state update** - объект, который обновляет состояние. Для начала опишем класс, который будет определять состояние системы:

In [40]:
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import StateGraph, START, END

class RecommenderState(TypedDict):
    message : str
    foodpref : str = ""
    winepref : str = ""
    maindish : str
    wine : str
    answer : str
    
def pr(state: RecommenderState):
    return ', '.join([f"{k}={v}" for k,v in state.items()])

Исходное сообщение пользователя будет в поле `message`, а финальный ответ - в поле `answer`. Начальные преференции пользователя будем хранить в полях `foodpref` и `winepref`, а выбранные блюдо и вино - в полях `maindish` и `wine`.

Логика работы будет такая:
* Вначале извлекаем предпочтения пользователя - за это отвечает функция `welcome` и агент-хостесс
* Основным узлом-маршрутизатором нашей многоагентной системы будет функция `clarify`. Если она видит какие-то из преференций `foodpref` или `winepref` - она выбирает по ним соответствующие блюда из меню с помощью агента-официанта.
* После `clarify` выполняется функция `route_user`, которая определяет, что делать дальше:
   - Если блюда и вино выбраны - мы отправляемся в узел `check_combination`, где с помощью сомелье проверяем, насколько выбранные блюдо и вино сочетаются между собой. Они могут не сочетаться, если пользователь сразу захотел есть стейк с белым вином.
   - Если какая-то из преференций `winepref` или `foodpref` не заполнены - происходит вызов функции `recommend_food` или `recommend_wine`, чтобы с помощью сомелье заполнить этот пробел
   - Если у пользователя нет преференций, то функция `select_random` выбирает для него какое-то блюдо, после чего выполнение агента продолжается с узла `recommend_wine`.

Помимо описания функций для каждого из узлов, нам нужно явным образом добавить узлы в граф с помощью `add_node`, и все переходы между ними, с помощью `add_edge` и `add_conditional_edges`. Дальше получишийся граф можно наприсовать.

In [41]:
from IPython.display import Image, display

def welcome_user(state: RecommenderState):
    print(f"> Welcome, state={pr(state)}")
    msg = state['message']
    print(f" + Сообщение от пользователя: {msg}")
    res = hostess(msg)
    print(f" + Получены преференции: {res}")
    return {
        'foodpref' : res.food_pref,
        'winepref' : res.wine_pref
    }
    
def route_user(state: RecommenderState) -> str:
    if state.get('maindish') and state.get('wine'): return "Done"
    if state['foodpref'] == "" and state['winepref'] == "": return "None"
    elif state['foodpref'] == "" and state['winepref'] != "": return "WantWine"
    elif state['foodpref'] != "" and state['winepref'] == "": return "WantFood"
    else: return "Both"

def select_random(state: RecommenderState):
    print(f"> SelectRandom, state={pr(state)}")
    maindish = waiter(
        "Порекомендуйте мне какое-нибудь одно основное блюдо. Напиши в ответ только название этого блюда.")
    print(f" + Выбрано случайное блюдо: {maindish}")
    return { "maindish" : maindish }
    
def recommend_food(state: RecommenderState):
    print(f"> Recommend Food, state={pr(state)}")
    wine = state.get("wine") or state.get("winepref")
    print(f" + Рекомендуем блюда к вину {wine}")
    foodpref = sommelier(f"Какие типы блюд подойдут к вину {wine}?")
    print(f" + Рекомендации: {foodpref}")
    return { "foodpref" : foodpref }

def recommend_wine(state: RecommenderState):
    print(f"> Recommend wine, state={pr(state)}")
    food = state.get('maindish') or state.get('foodpref')
    print(f" + Рекомендуем вино к блюду {food}")
    winepref = sommelier(f"Какие вина подойдут к блюду {food}?")
    print(f" + Рекомендации: {winepref}")
    return { "winepref" : winepref }
    
def clarify(state: RecommenderState):
    print(f"> Clarify, state={pr(state)}")
    upd = {}
    if state.get("winepref") and not state.get('wine'):
        print(f" + Подбираем вино: {state['winepref']}")
        upd["wine"] = waiter(f"Какое из следующих вин в меню подходит под такое описание: {state['winepref']}? Выбери только одно вино из меню.")
        print(f" + Выбрано вино: {upd['wine']}")
    if state.get("foodpref") and not state.get('maindish'):
        print(f" + Подбираем еду: {state['foodpref']}")
        upd["maindish"] = waiter(f"Какое из следующих блюд в меню подходит под такое описание: {state['foodpref']}?  Выбери только одно блюдо из меню.")
        print(f" + Выбрано блюдо: {upd['maindish']}")
    return upd

def check_combination(state: RecommenderState):
    print(f"> Check combination, state={pr(state)}")
    food = state["maindish"]
    wine = state["wine"]
    print(f" + Проверяем сочетаемость блюда {food} и вина {wine}")
    res = sommelier(f"Сочетается ли блюдо {food} с вином {wine}? Ответь ДА или НЕТ")
    if "нет" in res.lower():
        ans = waiter(f"Тебе хотят заказать следующее блюдо {state['maindish']} и вино {state['wine']}. Напиши вежливый ответ, что это блюдо и вино не очень сочетаются, и предложи выбрать другую комбинацию.")
    else:
        ans = waiter(f"Предложи гостям следующее блюдо {state['maindish']} и вино {state['wine']}.")
    return { "answer" : ans}

def yes_or_no(state: RecommenderState):
    if state["maindish"] and state["wine"]:
        return "Yes"
    else:
        return "No"

recommender_graph = StateGraph(RecommenderState)
recommender_graph.add_node("Welcome", welcome_user)
recommender_graph.add_node("SelectRandom", select_random)
recommender_graph.add_node("RecommendFood", recommend_food)
recommender_graph.add_node("RecommendWine", recommend_wine)
recommender_graph.add_node("CheckCombination", check_combination)
recommender_graph.add_node("Clarify", clarify)

recommender_graph.add_edge(START, "Welcome")
recommender_graph.add_edge("Welcome", "Clarify")
recommender_graph.add_conditional_edges(
    "Clarify",
    route_user,
    {
        "None": "SelectRandom",
        "WantWine" : "RecommendFood",
        "WantFood" : "RecommendWine",
        "Both" : "CheckCombination",
        "Done" : "CheckCombination"
    })
recommender_graph.add_edge("SelectRandom", "RecommendWine")
recommender_graph.add_edge("RecommendFood", "Clarify")
recommender_graph.add_edge("RecommendWine", "Clarify")
recommender_graph.add_edge("CheckCombination", END)

compiled_graph = recommender_graph.compile()

#display(Image(compiled_graph.get_graph().draw_mermaid_png(max_retries=5,retry_delay=2.0)))

Для того, чтобы "поговорить" с этой системой, дадим ей на вход начальное состояние:

In [42]:
res = compiled_graph.invoke(
    {
        "message" : "Привет, я хочу поесть что-то из мяса"
    }
)
res

> Welcome, state=message=Привет, я хочу поесть что-то из мяса
 + Сообщение от пользователя: Привет, я хочу поесть что-то из мяса
 + Получены преференции: food_pref='мясо' wine_pref=''
> Clarify, state=message=Привет, я хочу поесть что-то из мяса, foodpref=мясо, winepref=
 + Подбираем еду: мясо
 + Выбрано блюдо: Под описание "мясо" лучше всего подходит **Стейк "Бык на взводе"** — сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей, с золотой солью Гималаев.
> Recommend wine, state=message=Привет, я хочу поесть что-то из мяса, foodpref=мясо, winepref=, maindish=Под описание "мясо" лучше всего подходит **Стейк "Бык на взводе"** — сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей, с золотой солью Гималаев.
 + Рекомендуем вино к блюду Под описание "мясо" лучше всего подходит **Стейк "Бык на взводе"** — сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей, с золотой солью Гималаев.
 + Рекомендации: К стейку **"Бык на взводе"** (сочному р

{'message': 'Привет, я хочу поесть что-то из мяса',
 'foodpref': 'мясо',
 'winepref': 'К стейку **"Бык на взводе"** (сочному рибай с розовым сердцем) отлично подойдут следующие вина:\n\n1. **Каберне Совиньон** от **Château de Parenchère, Франция**, 2017 года — вино с насыщенным танинным каркасом, яркой кислотностью и богатым букетом спелой вишни, табака, дуба и чёрного перца. Оно идеально дополняет насыщенный вкус стейка.\n\n2. **Пино Нуар** от **Domaine de la Romanée-Conti, Франция**, 2017 года — более утончённый выбор с нежной танинностью и ароматами вишни, красной смородины и фиалки. Подойдёт, если вы предпочитаете более изысканное и элегантное сочетание.\n\nОба вина подчеркнут сочность и глубину вкуса стейка, но Каберне Совиньон — более классический и мощный выбор.',
 'maindish': 'Под описание "мясо" лучше всего подходит **Стейк "Бык на взводе"** — сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей, с золотой солью Гималаев.',
 'wine': 'Из представленных в меню в

In [43]:
printx(res['answer'])

Рекомендую вам попробовать **стейк "Бык на взводе"** — сочный рибай с розовым сердцем, томлёный в дыму аргентинских страстей и поданный с золотой солью Гималаев. Это настоящий триумф вкуса и текстуры.

В паре с ним идеально сочетается **Каберне Совиньон от Château Lafite Rothschild, Франция, 2018 года** — вино с насыщенным танинным каркасом, яркой кислотностью и богатым букетом спелой вишни, табака, дуба и чёрного перца. Оно подчёркивает глубину и насыщенность стейка, создавая гармоничное и запоминающееся сочетание.

Приятного аппетита!

Рассмотрим пример, когда пользователь хочет изначально не слишком хорошее сочетание:

In [44]:
res = compiled_graph.invoke(
    {
        "message" : "Привет, я хочу съесть какое-то рыбное блюдо с мерло!"
    }
)
printx(res['answer'])

> Welcome, state=message=Привет, я хочу съесть какое-то рыбное блюдо с мерло!
 + Сообщение от пользователя: Привет, я хочу съесть какое-то рыбное блюдо с мерло!
 + Получены преференции: food_pref='рыбное блюдо' wine_pref='мерло'
> Clarify, state=message=Привет, я хочу съесть какое-то рыбное блюдо с мерло!, foodpref=рыбное блюдо, winepref=мерло
 + Подбираем вино: мерло
 + Выбрано вино: Вино, подходящее под описание "Мерло", — **Мерло от Marchesi Antinori, Италия, 2019 года**, цена за бокал — **2800 рублей**.
 + Подбираем еду: рыбное блюдо
 + Выбрано блюдо: Блюдо, подходящее под описание "рыбное блюдо", — **"Лосось в мечтах о Норвегии"**: нежнейшее филе-кусок, запечённое под корочкой из "загадочных северных трав" (укропа), с лимонным бризом.
> Check combination, state=message=Привет, я хочу съесть какое-то рыбное блюдо с мерло!, foodpref=рыбное блюдо, winepref=мерло, maindish=Блюдо, подходящее под описание "рыбное блюдо", — **"Лосось в мечтах о Норвегии"**: нежнейшее филе-кусок, запечённ

Спасибо за интерес к нашему меню!  

Блюдо **"Лосось в мечтах о Норвегии"** — это нежное, изысканное рыбное блюдо с лёгкой кислинкой лимона и ароматом укропа, и оно лучше всего раскрывается с белыми винами, например, с **Совиньон Блан от Cloudy Bay, Новая Зеландия, 2021 года** — его свежесть и цитрусовые нотки идеально дополнят вкус лосося.

К сожалению, **Мерло от Marchesi Antinori** — это мягкое, но насыщенное красное вино с телесной структурой, которое может перебить тонкий вкус рыбы. Оно прекрасно сочетается с мясными блюдами, но с лососем — не лучший выбор.

Позвольте предложить вам более гармоничную пару:  
**"Лосось в мечтах о Норвегии"** + **Совиньон Блан (Cloudy Bay, 2021)** — это настоящий дуэт севера и свежести.

Буду рад помочь с оформлением заказа!

И наконец случай, когда у пользователя нет предпочтений:

In [45]:
res = compiled_graph.invoke(
    {
        "message" : "Что вкусного есть у вас сегодня?"
    }
)
printx(res['answer'])

> Welcome, state=message=Что вкусного есть у вас сегодня?
 + Сообщение от пользователя: Что вкусного есть у вас сегодня?
 + Получены преференции: food_pref='' wine_pref=''
> Clarify, state=message=Что вкусного есть у вас сегодня?, foodpref=, winepref=
> SelectRandom, state=message=Что вкусного есть у вас сегодня?, foodpref=, winepref=
 + Выбрано случайное блюдо: Стейк "Бык на взводе"
> Recommend wine, state=message=Что вкусного есть у вас сегодня?, foodpref=, winepref=, maindish=Стейк "Бык на взводе"
 + Рекомендуем вино к блюду Стейк "Бык на взводе"
 + Рекомендации: К стейку **"Бык на взводе"** отлично подойдут следующие вина:

1. **Каберне Совиньон от Château Lafite Rothschild, Франция, 2018 года** — вино с насыщенным танинным каркасом, глубоким букетом спелой вишни, табака, дуба и чёрного перца. Его структура идеально сбалансирована с сочной, дымной текстурой стейка.

2. **Пино Нуар от Domaine de la Romanée-Conti, Франция, 2017 года** — более изысканный и элегантный выбор с нежными н

Позвольте предложить вам настоящий дуэт силы и элегантности — **стейк "Бык на взводе"**: сочный рибай с розовым сердцем, томлёный в дыму аргентинских страстей и поданный с золотой солью Гималаев.

В идеальной паре с ним — **Каберне Совиньон от Château Lafite Rothschild, Франция, 2018 года**. Это вино с насыщенным танинным каркасом, глубоким ароматом спелой вишни, табака, дуба и чёрного перца создаёт гармоничное и мощное сочетание, подчёркивающее богатый вкус стейка.

Приятного аппетита и вдохновлённого ужина!

## Тестируем качество ответов: RAGAS

Для того, чтобы отлаживать и улучшать многоагентные и RAG-системы, необходимо научиться автоматически измерять какую-то меру качества ответа. Тестирование диалоговых систем - вещь непростая, по нескольким причинам:

* Непонятно, как оценивать качество естественно-языкового ответа
* Сложно тестировать многоступенчатый диалог (multi-turn conversation)

Для решения этих проблем сущестует библиотека [RAGAS](https://ragas.io). Рассмотрим её использование на примере тестирования нашей системы.

Возможны разные подходы к тестированию (которые поддерживаются RAGAS):
* Мы вручную создаём датасет с вопросами и идеальными вариантами ответа
* Мы создаём датасет вопросов, и проверяем их по некоторому набору критериев или с помощью LLM в качества арбитра (LLM as a Judge)
* Датасет вопросов автоматически создаётся по текстовой базе знаний RAG. RAGAS содержит развитые средства генерации вопросов, включающие построение графа знаний и различные его трансформации для получения разнообразных и нетривиальных вопросов.


Рассмотрим первый подход. В этом случае нам понадобится датасет с вопросами-ответами:

In [4]:
import json

ds = json.load(open('data/eval/evaluate_advisor.json','r'))
ds

[{'user_input': 'Я хочу съесть что-то из мяса',
  'reference': 'Я рекомендую сочный стейк и терпкое красное вино, например, Мальбек или Мерло.'},
 {'user_input': 'По четвергам предпочитаю рыбу',
  'reference': 'Я рекомендую рыбное блюдо, например, лосось на гриле, и сухое белое вино, например, Совиньон Блан или Рислинг.'},
 {'user_input': 'Я хочу съесть лосося с красным вином',
  'reference': 'Сожалею, но мы не рекомендуем есть рыбу с красным вином.'}]

Теперь возьмём этот датасет, запустим нашу многоагентную систему для каждого из вопросов, и запомним ответ в поле `response`:

In [ ]:
for i,x in enumerate(ds):
    print(f"==== RUNNING EXAMPLE {i+1}: {x['user_input']}")
    res = compiled_graph.invoke(
    {
        "message" : x['user_input']
    })
    ds[i]['response'] = res['answer']


In [21]:
with open('data/eval/evaluate_advisor_results.json','w') as f:
    json.dump(ds,f,indent=4, ensure_ascii=False)

Для ускорения демонстрации можно пропустить предыдущий шаг и сразу загрузить датасет с ответами системы:

In [5]:
ds = json.load(open('data/eval/evaluate_advisor_results.json','r'))
ds

[{'user_input': 'Я хочу съесть что-то из мяса',
  'reference': 'Я рекомендую сочный стейк и терпкое красное вино, например, Мальбек или Мерло.',
  'response': 'Стейк «Бык на взводе» — это сочный рибай с розовым сердцем, томлёный в дыме аргентинских страстей, который подаётся с золотой солью Гималаев. К этому блюду может отлично подойти вино Мерло от Marchesi Antinori, Италия. Это элегантное и выдержанное вино, которое хорошо сочетается со стейком. Цена стейка составляет 2500 рублей, а цена бокала вина — 2800 рублей.'},
 {'user_input': 'По четвергам предпочитаю рыбу',
  'reference': 'Я рекомендую рыбное блюдо, например, лосось на гриле, и сухое белое вино, например, Совиньон Блан или Рислинг.',
  'response': 'К вашему столу я могу предложить блюдо «Лосось в мечтах о Норвегии» — это нежный филе-кусок, запечённый под корочкой из «загадочных северных трав» (укропа), который подаётся с лимонным бризом. В качестве дополнения к этому блюду могу порекомендовать вино Совиньон Блан от Cloudy Bay

Теперь нам нужно оценить близость ответа системы к идеальному ответу. Для этого можно использовать текстовые эмбеддинги и метрику семантической близости. RAGAS позволяет нам использовать эмбеддинги из библиотеки LangChain: 

In [6]:
from ragas.dataset_schema import EvaluationDataset, SingleTurnSample
from ragas import evaluate
#from ragas.embeddings import OpenAIEmbeddings
from ragas.metrics import answer_similarity
from langchain_community.embeddings.yandex import YandexGPTEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

emodel = f"emb://{folder_id}/text-search-query/latest"

#embeddings = OpenAIEmbeddings(aclient,model=emodel)
embeddings = YandexGPTEmbeddings(folder_id=folder_id,iam_token=api_key)
embeddings = LangchainEmbeddingsWrapper(embeddings)

tests = EvaluationDataset([ SingleTurnSample(**x) for x in ds ])
evaluate(tests,[ answer_similarity ], embeddings=embeddings)


C:\Users\dmitr\AppData\Local\Temp\ipykernel_65188\4089688648.py:12: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  embeddings = LangchainEmbeddingsWrapper(embeddings)


Evaluating:   0%|          | 0/3 [00:00<?, ?it/s]

{'answer_similarity': 0.7233}

В RAGAS за проверку разных аспектов качества отвечают т.н. **метрики**. Выше мы использовали встроенную метрику `answer_similarity`, основанную на эмбеддингах.

Второй вариант проверки ответов - это использование LLM as a Judge. Есть какое-то количество встроенных метрик с предопределёнными промптами (например, `AnswerAccuracy`), либо же можно определять свои метрики и задавать им промпты явно.

Например, попробуем оценить правильность рекомендаций вина и еды в соответствии с простым правилом "красное - к мясу, белое - к рыбе":

In [17]:
from yandex_cloud_ml_sdk import AsyncYCloudML
from ragas.metrics import AspectCritic, AnswerAccuracy
from ragas.llms import LangchainLLMWrapper,llm_factory
from langchain_openai import OpenAI

#lc_client=OpenAI(
#    model = model,
#    base_url = "https://rest-assistant.api.cloud.yandex.net/v1/",
#    api_key = api_key
#)

asdk = AsyncYCloudML(folder_id=folder_id,auth=api_key)
judge_llm = LangchainLLMWrapper(asdk.models.completions("yandexgpt", model_version="rc").langchain())

match_criteria = """
Подходит ли рекомендованное вино к блюду? Красные вина подходят к мясным блюдам,
белые - к рыбным. Если пользователь хочет несочетаемую комбинацию - об этом
нужно явно сказать в ответе, что такая комбинация не рекомендуется.
"""
wine_food_match = AspectCritic("wine_food_match",match_criteria,llm=judge_llm)
wine_food_match.async_mode=False
answer_accuracy = AnswerAccuracy(llm=judge_llm)
answer_accuracy.async_mode=False

evaluate(tests,[ wine_food_match, answer_accuracy ], llm=judge_llm)

Evaluating:   0%|          | 0/6 [00:00<?, ?it/s]

{'wine_food_match': 1.0000, 'nv_accuracy': 0.5000}

Установив таким образом механизм автоматического измерения и отслеживания метрик, мы теперь можем изменять параметры модели, экспериментировать с промптами и т.д. для оптимизации результата.

## Удаляем лишнее

После демонстрации мы можем удалить все ассистенты, файлы, индексы и переписки.

**ВНИМАНИЕ:** Код ниже удаляет все ассистенты, файлы, индексы и переписки в каталоге, с которым работает Yandex Cloud ML SDK. Если у вас есть другие проекты, использующие ассистентов, то лучше не выполняйте этот код, а дождитесь, когда объекты будут сами удалены по TTL.

In [18]:
vector_stores = client.vector_stores.list()
for v in vector_stores:
    print(f" + Deleting vector store id={v.id}")
    client.vector_stores.delete(vector_store_id=v.id)

files = client.files.list(purpose='assistants')
for f in files:
    print(f" + Deleting file id={f.id}")
    client.files.delete(file_id=f.id)

 + Deleting vector store id=fvt2fnc2uaoters66s82
 + Deleting vector store id=fvtiqod24d1p7d7akla3
 + Deleting vector store id=fvtd69ju2e43u1ir70r5
 + Deleting vector store id=fvtf6j9kh6a72829g4ki
 + Deleting vector store id=fvtsi00cb7nofnt6dfq1
 + Deleting file id=fvthbunb4oiluq9lifgm
 + Deleting file id=fvtto04aemtvre337nbb
 + Deleting file id=fvttct3h4h17u9qq68tt
 + Deleting file id=fvt2m3skfa6sj2a0tg7k
 + Deleting file id=fvtrmqdl2f3bc8mf6anm
 + Deleting file id=fvt0q3cugcaa8egd0crk
 + Deleting file id=fvt6p32d9ms4cj1m9mg4
 + Deleting file id=fvt52b7umgta5ilceos6
 + Deleting file id=fvtbi1fr84q03dh2okal
 + Deleting file id=fvtsnnmhpecmntdgq608
 + Deleting file id=fvtbb609gqmfp0bn03e4
 + Deleting file id=fvtj7a8cnpgamsfotine
 + Deleting file id=fvt7fsg1geltt3top7e0
 + Deleting file id=fvt3jp1ulqemv4deosn8
 + Deleting file id=fvtimhsrcndl3eaiqpeb
 + Deleting file id=fvtvv498344lojnh50uh
 + Deleting file id=fvt5g1vur30uvq86qjpm
 + Deleting file id=fvtti0u5ov65p8ralpf6
 + Deleting file 

## Выводы

LangGraph - это фреймворк, позволяющий выстраивать заранее спланированные и управляемые графы взаимодействия между агентами. Это чуть более трудоёмко, зато мы контролируем процесс взаимодействия агентов, что часто бывает полезно при создании реальных систем.